In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

# import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
# pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'predict')
sys.path.append(source_path)
import matplotlib.pyplot as plt

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    get_predict_loader,
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    dataloader_test_by_stock
)
from metrics import calc_spread_return_sharpe
from predict import run_pred_step
print(torch.__version__)

import pickle

import warnings
warnings.filterwarnings("ignore")

C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.11.0


# Get data

In [2]:
FOLDER = 'example_test_files'
df = get_data(folder=FOLDER)
# df['Target'] = df['Close'].pct_change()
# df = df.sort_values(by=['RowId', 'Date'])
print(df.head())

CONT_COLS=['Close', 'Open', 'High', 'Low']
TS_IN_FEATURES = len(CONT_COLS)
CAT_FEATURES = 4 #cat.shape[1]
print('CAT_FEATURES:', CAT_FEATURES)
EMBEDDING_DIM = 300
NO_EMBEDDING = 2000 #2 * len(df_train_cat)

                    RowId  SecuritiesCode    Open    High     Low   Close  \
Date                                                                        
2021-12-06  20211206_1301            1301  2982.0  2982.0  2965.0  2971.0   
2021-12-06  20211206_1332            1332   592.0   599.0   588.0   589.0   
2021-12-06  20211206_1333            1333  2368.0  2388.0  2360.0  2377.0   
2021-12-06  20211206_1375            1375  1230.0  1239.0  1224.0  1224.0   
2021-12-06  20211206_1376            1376  1339.0  1372.0  1339.0  1351.0   

             Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  
Date                                                                      
2021-12-06     8900               1.0               NaN            False  
2021-12-06  1360800               1.0               NaN            False  
2021-12-06   125900               1.0               NaN            False  
2021-12-06    81100               1.0               NaN            False  
2021-12-06

In [3]:
df['SecuritiesCode'].unique()

array([1301, 1332, 1333, ..., 9993, 9994, 9997], dtype=int64)

In [4]:
df[df['SecuritiesCode'] == 1301]

,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
Date,,,,,,,,,,
2021-12-06,20211206_1301,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
2021-12-07,20211207_1301,1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,NaN,False


In [5]:
stocks = df['SecuritiesCode'].unique()
count = 0
BATCH_SIZE = 512
weight_decay = 0.01

# model = create_model()
# trainer = Trainer(model, optimizer_name='rmsprop', lr=1.3333e-4, weight_decay=weight_decay)
# torch.autograd.set_detect_anomaly(True)
trained_model = torch.load('./trained_model.pt')
with open('./scaler_dict.pkl', 'rb') as f:
    scaler_dict = pickle.load(f)


for stock in stocks:
    teast_dataloader = None
    if count > 15:
        break
    try:
        print(f'Run prediction for stock: {stock}')
        transformer = scaler_dict[stock]
        test_dataloader = dataloader_test_by_stock(
            df, 
            stock, 
            transformer=transformer,
            batch_size=BATCH_SIZE, 
            continous_cols=CONT_COLS,
            target_col=None
        )
        run_pred_step(test_dataloader, trained_model, x_cat=True, target=None)
        count += 1
    except Exception as e:
        print(f'Exception {e}')

    

Run prediction for stock: 1301
continuos shape: (2, 8)  categorical shape: (2, 4)
Test predictions:
tensor([[-0.0051],
        [-0.0044]], device='cuda:0', grad_fn=<AddBackward0>)
Run prediction for stock: 1332
continuos shape: (2, 8)  categorical shape: (2, 4)
Test predictions:
tensor([[-0.0051],
        [-0.0045]], device='cuda:0', grad_fn=<AddBackward0>)
Run prediction for stock: 1333
continuos shape: (2, 8)  categorical shape: (2, 4)
Test predictions:
tensor([[-0.0051],
        [-0.0044]], device='cuda:0', grad_fn=<AddBackward0>)
Run prediction for stock: 1375
Exception 1375
Run prediction for stock: 1376
continuos shape: (2, 8)  categorical shape: (2, 4)
Test predictions:
tensor([[-0.0050],
        [-0.0044]], device='cuda:0', grad_fn=<AddBackward0>)
Run prediction for stock: 1377
continuos shape: (2, 8)  categorical shape: (2, 4)
Test predictions:
tensor([[-0.0051],
        [-0.0044]], device='cuda:0', grad_fn=<AddBackward0>)
Run prediction for stock: 1379
continuos shape: (2, 8)

# Load model